Here, we'll prepare the data for visualization of popular topic distributions across some countries.

In [1]:
from gensim import corpora, models, similarities, matutils
import itertools
import numpy as np
import pandas as pd
from unidecode import unidecode

In [2]:
dropbox = "/Users/WSKIM/Dropbox/moviemeta/"

We can construct a topic matrix from imdb data, as follows:

In [3]:
imdb_lda = models.LdaModel.load(dropbox +'lda_imdb.model')
imdb_corpus = corpora.MmCorpus(dropbox +'lda_imdb.corpus')
imdb_dict = corpora.Dictionary.load(dropbox +'lda_imdb.dict')
imdb_meta_df = pd.read_csv(dropbox + 'imdb_meta_df.csv')
imdb_topic_matrix = matutils.corpus2dense(imdb_lda[imdb_corpus], num_terms=30)
imdb_topic_df = pd.DataFrame(np.ndarray.transpose(imdb_topic_matrix))
imdb_topic_df = pd.concat([imdb_topic_df, imdb_meta_df], axis=1) 
imdb_topic_df['title'] = imdb_topic_df['title'].apply(unidecode)
imdb_topic_df['title'] = imdb_topic_df['title'].apply(lambda x : x.split('(')[0].strip())
imdb_meta_df['title'] = imdb_meta_df['title'].apply(unidecode)
imdb_meta_df['title'] = imdb_meta_df['title'].apply(lambda x : x.split('(')[0].strip())

C:\Anaconda\lib\site-packages\pandas\core\series.py:2169: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [4]:
imdb_topic_df.shape

(259028, 33)

Now we have dataframe of topic matrix for all movies in imdb : column 0 ~29 represents 30 topics, and values in each column indicates the relative prevalence of that topic in each movie. 

In [22]:
imdb_topic_df.head(5)
#imdb_topic_df.count() - number of not NaN values = 199795 movies (can be checked by imdb_topic_df.tail())

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,Unnamed: 0,title,year
0,0,0,0,0,0.000000,0.102051,0,0.000000,0,0,...,0.000000,0.654931,0.000000,0.000000,0.000000,0,0,0,#1 Cheerleader Camp,2010
1,0,0,0,0,0.037337,0.000000,0,0.800649,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,#1 Serial Killer,2013
2,0,0,0,0,0.000000,0.000000,0,0.000000,0,0,...,0.000000,0.229884,0.000000,0.000000,0.069421,0,0,2,#1 at the Apocalypse Box Office,2015
3,0,0,0,0,0.000000,0.291748,0,0.290483,0,0,...,0.000000,0.000000,0.035630,0.030289,0.000000,0,0,3,#137,2011
4,0,0,0,0,0.000000,0.000000,0,0.516780,0,0,...,0.224002,0.000000,0.107093,0.000000,0.000000,0,0,4,#29,2012


Let's calculate the topic_score for each topic and select global top 10 topics, across all movies. 

In [20]:
df_sub = imdb_topic_df.iloc[:,:30]
topic_score = df_sub.sum(axis=0).tolist()
topic_score[:5], len(topic_score)

([1858.8319, 6125.7363, 1851.713, 7197.4927, 1795.5492], 30)

In [36]:
df_matrix = pd.DataFrame(imdb_topic_matrix) # row = topic, column = movie
df_matrix['topic_score'] = pd.Series(topic_score, index=df_matrix.index)
#df_matrix.head(6)

,0,1,2,3,4,5,6,7,8,9,...,199786,199787,199788,199789,199790,199791,199792,199793,199794,topic_score
0,0.000000,0.000000,0,0.000000,0,0.018813,0.000000,0.000000,0.036073,0.000000,...,0.00000,0.035759,0.019056,0.000000,0.092732,0,0,0,0.000000,1858.831909
1,0.000000,0.000000,0,0.000000,0,0.173562,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.025737,0.000000,0.046417,0,0,0,0.000000,6125.736328
2,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.024270,...,0.00000,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,1851.713013
3,0.000000,0.000000,0,0.000000,0,0.000000,0.178128,0.000000,0.000000,0.076878,...,0.00000,0.000000,0.027760,0.000000,0.000000,0,0,0,0.086073,7197.492676
4,0.000000,0.037337,0,0.000000,0,0.000000,0.147059,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,1795.549194
5,0.102051,0.000000,0,0.291748,0,0.000000,0.233787,0.093865,0.213503,0.303941,...,0.45848,0.000000,0.078271,0.193266,0.000000,0,0,0,0.000000,17746.820312


In [71]:
#Global top 10 topics
topics =['love, family', 'love, relationships', 'earth, nature', 'friendships, relationships', 'crime, mistery', 'society, culture', 
         'home', 'money', 'gangs, drug, police', 'documentary, interview', 'ight life, enjoyment', 'music', 'war', 
         'school, college', 'fantasy, fairy tale', 'cowboys and indians', 'photagraphy, design', 'comedy', 'spies, terrorism', 
         'crime, police, underworld', 'places, nature, scenery', 'historical, costumes','town','hollywood, stars', 'press, politics','farming, country side',
         'fantasy, christmas', 'magic, myths', 'ships, sailing, pirates', 'mixed']
df_matrix['label'] = pd.Series(topics, index= df_matrix.index) 
global_top = df_matrix.sort_values('topic_score', axis=0, ascending=False, inplace=False)
global_top.head(10) #topic 5, 28, 21, 7,24, 11, 20, 29, 13, 25

,0,1,2,3,4,5,6,7,8,9,...,199787,199788,199789,199790,199791,199792,199793,199794,topic_score,label
5,0.102051,0.000000,0.000000,0.291748,0.000000,0.000000,0.233787,0.093865,0.213503,0.303941,...,0.000000,0.078271,0.193266,0.000000,0.000000,0.000000,0.000000,0.000000,17746.820312,"society, culture"
28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.263698,0.000000,0.000000,0.000000,...,0.143780,0.376891,0.307183,0.419190,0.481849,0.000000,0.000000,0.094967,16824.498047,"ships, sailing, pirates"
21,0.000000,0.000000,0.187699,0.000000,0.000000,0.300146,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.319119,0.465730,0.101383,0.097219,15152.167969,"historical, costumes"
7,0.000000,0.800649,0.000000,0.290483,0.516780,0.000000,0.092949,0.201333,0.000000,0.200787,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.047477,0.704598,0.000000,14722.139648,money
24,0.654931,0.000000,0.229884,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10737.474609,"press, politics"
11,0.108345,0.000000,0.000000,0.000000,0.000000,0.054433,0.000000,0.401697,0.096331,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080039,0.000000,10379.306641,music
20,0.000000,0.129871,0.479663,0.000000,0.000000,0.115910,0.000000,0.000000,0.044879,0.077263,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068366,0.000000,8557.800781,"places, nature, scenery"
29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051851,...,0.200441,0.122704,0.000000,0.237313,0.000000,0.057303,0.000000,0.181364,8418.423828,mixed
13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067230,0.000000,0.000000,...,0.206017,0.107891,0.403261,0.000000,0.000000,0.000000,0.000000,0.000000,7806.116699,"school, college"
25,0.000000,0.000000,0.000000,0.035630,0.107093,0.000000,0.000000,0.000000,0.000000,0.110926,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7438.209473,"farming, country side"


In [6]:
#Top movies for 6 countries (US, GE, UK, South Korea, Japan, Hong Kong) 
topmovies_df=pd.read_csv(dropbox + "TopMovies_final.csv")

In [58]:
#Top movies for each country
kr_df = topmovies_df[topmovies_df.country=="South Korea"] #available since 2007
jp_df = topmovies_df[topmovies_df.country=="Japan"] #available since 2002
hk_df = topmovies_df[topmovies_df.country=="Hong Kong"] #available since 2002
uk_df = topmovies_df[topmovies_df.country=="United Kingdom"] #available since 2002
ge_df = topmovies_df[topmovies_df.country=="Germany"] #available since 2002
us_df = topmovies_df[topmovies_df.country=="United States"] #available since 2001
kr_df.shape, jp_df.shape, hk_df.shape, uk_df.shape, ge_df.shape, us_df.shape

((2553, 3), (1300, 3), (1300, 3), (4781, 3), (4139, 3), (6835, 3))

In [64]:
#collect movies data from imdb_topic_df : check the minimum number of available data, since 2007
years = range(2007, 2015)
available_kr =[]
available_jp =[]
available_hk =[]
available_uk =[]
available_ge =[]
available_us =[]
for year in years:
    num_movies_kr = imdb_meta_df[imdb_meta_df['year']==year][imdb_meta_df['title'].isin(kr_df[kr_df['year']==year].movie.tolist())]
    available_kr.append(num_movies_kr.shape)
    num_movies_jp = imdb_meta_df[imdb_meta_df['year']==year][imdb_meta_df['title'].isin(jp_df[jp_df['year']==year].movie.tolist())]
    available_jp.append(num_movies_jp.shape)
    num_movies_hk = imdb_meta_df[imdb_meta_df['year']==year][imdb_meta_df['title'].isin(hk_df[hk_df['year']==year].movie.tolist())]
    available_hk.append(num_movies_hk.shape)
    num_movies_uk = imdb_meta_df[imdb_meta_df['year']==year][imdb_meta_df['title'].isin(uk_df[uk_df['year']==year].movie.tolist())]
    available_uk.append(num_movies_uk.shape)
    num_movies_ge = imdb_meta_df[imdb_meta_df['year']==year][imdb_meta_df['title'].isin(ge_df[ge_df['year']==year].movie.tolist())]
    available_ge.append(num_movies_ge.shape)
    num_movies_us = imdb_meta_df[imdb_meta_df['year']==year][imdb_meta_df['title'].isin(us_df[us_df['year']==year].movie.tolist())]
    available_us.append(num_movies_us.shape)
print "Available movies for South Korea : " ,available_kr
print "Available movies for Japan : " ,available_jp
print "Available movies for Hong Kong : " ,available_hk
print "Available movies for United Kingdom : " ,available_uk
print "Available movies for Germany : " ,available_ge
print "Available movies for United States : " ,available_us

Available movies for South Korea :  [(74, 3), (68, 3), (63, 3), (65, 3), (65, 3), (51, 3), (50, 3), (51, 3)]
Available movies for Japan :  [(29, 3), (31, 3), (23, 3), (30, 3), (23, 3), (24, 3), (23, 3), (27, 3)]
Available movies for Hong Kong :  [(45, 3), (43, 3), (45, 3), (41, 3), (43, 3), (41, 3), (42, 3), (57, 3)]
Available movies for United Kingdom :  [(189, 3), (192, 3), (142, 3), (132, 3), (192, 3), (198, 3), (169, 3), (169, 3)]
Available movies for Germany :  [(124, 3), (105, 3), (104, 3), (97, 3), (110, 3), (117, 3), (107, 3), (123, 3)]
Available movies for United States :  [(226, 3), (220, 3), (208, 3), (200, 3), (197, 3), (191, 3), (179, 3), (174, 3)]


As we can see from above results, data for Asian countires are not enough. Thus, it would be better to study South Korea, United States, Germany, and United Kingdom : 50 movies for each year since 2007, because the number of movies should be same for all countries to see if there is any similarity or difference in movie topics distribution for those countries. It's very small number of movies...is it meaningful to analyze?? 

In [72]:
def get(df, year, country=None):
    df = df[df['year']==year]
    if country:
        return df[df['country']==country]
    return df

imdb_merged_df = pd.DataFrame()

for year in range(2001, 2015):
    for country in ['Germany','United States', 'United Kingdom', 'South Korea', 'Japan', 'Hong Kong']:
        topic = get(imdb_topic_df, year)
        top = get(topmovies_df, year,country)
        #remove date from imdb title
        titles = topic.title.apply(lambda x : x.split('(')[0].strip())
        topic = topic[titles.isin(top.movie.tolist())]
        topic['country'] = country
        imdb_merged_df = pd.concat([imdb_merged_df, topic])
imdb_merged_df.head()    

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,Unnamed: 0,title,year,country
5579,0.000000,0,0.000000,0.239655,0.000000,0.064387,0.031102,0.000000,0.000000,0.055464,...,0.000000,0.079102,0.085095,0.075371,0.000000,0.040585,5579,A Knight's Tale,2001,Germany
13945,0.000000,0,0.029535,0.000000,0.000000,0.150826,0.000000,0.070662,0.000000,0.000000,...,0.149596,0.000000,0.000000,0.000000,0.046496,0.000000,13945,Along Came a Spider,2001,Germany
14601,0.000000,0,0.027998,0.000000,0.090976,0.000000,0.130325,0.000000,0.027825,0.000000,...,0.070552,0.000000,0.015746,0.061719,0.000000,0.173727,14601,America's Sweethearts,2001,Germany
14900,0.000000,0,0.000000,0.000000,0.000000,0.107158,0.000000,0.663062,0.000000,0.000000,...,0.000000,0.062903,0.038708,0.000000,0.000000,0.097305,14900,American Pie 2,2001,Germany
16626,0.029772,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.030694,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.387321,0.000000,16626,Angel Eyes,2001,Germany


In [139]:
#filtering all movies available in imdb for each country
imdb_df_us_all = imdb_merged_df[imdb_merged_df['country']=='United States']
imdb_df_ge_all = imdb_merged_df[imdb_merged_df['country']=='Germany']
imdb_df_uk_all = imdb_merged_df[imdb_merged_df['country']=='United Kingdom']
imdb_df_kr_all = imdb_merged_df[imdb_merged_df['country']=='South Korea']
imdb_df_us_all.shape, imdb_df_ge_all.shape, imdb_df_uk_all.shape, imdb_df_kr_all.shape

((2859, 34), (1451, 34), (2045, 34), (487, 34))

Because all movie plots are not available in imdb, we have to collect 50 available movie plots for each year and each country, while maintaining the rank(50 movies from the top).

In [ ]:
#selecting 50 movies for each year in each country, for those imdb data are available
#South Korea - 2014 : head(167), 2013: 185, 2012 : 168, 2011 : 155, 2010 : 150, 2009: 160, 2008 : 135, 2007 : 142
#United States - 2014 : head(71), 2013:76 , 2012 :61 , 2011 :66 , 2010 : 64, 2009: 59 , 2008 : 59 , 2007 : 57 
#United Kingdom - 2014 : head(72), 2013: 84, 2012 :74 , 2011 :74 , 2010 :74 , 2009:82, 2008 :71, 2007 :67 
#Germany - 2014 : head(82), 2013:103, 2012 :92 , 2011 :91 , 2010 :90 , 2009:101, 2008 :89, 2007 :89
yearly_tops = ge_df[ge_df['year']==2014].head(82)
ge_tops_title = yearly_tops['movie'].tolist()
imdb_df_ge_tops = imdb_df_ge_all[imdb_df_ge_all['title'].isin(ge_tops_title)]
imdb_df_ge_tops.shape[0]

In [287]:
#list of titles of available movies for each country, 2007~2014
yearly_tops_2014 = ge_df[ge_df['year']==2014].head(82)
tops_2014 = yearly_tops_2014['movie'].tolist()
yearly_tops_2013 = ge_df[ge_df['year']==2013].head(103)
tops_2013 = yearly_tops_2013['movie'].tolist()   
yearly_tops_2012 = ge_df[ge_df['year']==2012].head(92)
tops_2012 = yearly_tops_2012['movie'].tolist()   
yearly_tops_2011 = ge_df[ge_df['year']==2011].head(91)
tops_2011 = yearly_tops_2011['movie'].tolist()   
yearly_tops_2010 = ge_df[ge_df['year']==2010].head(90)
tops_2010 = yearly_tops_2010['movie'].tolist()   
yearly_tops_2009 = ge_df[ge_df['year']==2009].head(101)
tops_2009 = yearly_tops_2009['movie'].tolist()   
yearly_tops_2008 = ge_df[ge_df['year']==2008].head(89)
tops_2008 = yearly_tops_2008['movie'].tolist()   
yearly_tops_2007 = ge_df[ge_df['year']==2007].head(89)
tops_2007 = yearly_tops_2007['movie'].tolist()   
ge_tops = tops_2014 + tops_2013 + tops_2012 + tops_2011 + tops_2010 + tops_2009 + tops_2008 + tops_2007
len(ge_tops)

737

In [288]:
#filtering imdb data for each country
imdb_kr_df_tops = imdb_df_kr_all[imdb_df_kr_all['title'].isin(kr_tops)]
imdb_us_df_tops = imdb_df_us_all[imdb_df_us_all['title'].isin(us_tops)]
imdb_uk_df_tops = imdb_df_uk_all[imdb_df_uk_all['title'].isin(uk_tops)]
imdb_ge_df_tops = imdb_df_ge_all[imdb_df_ge_all['title'].isin(ge_tops)]
imdb_kr_df_tops.shape, imdb_us_df_tops.shape, imdb_uk_df_tops.shape, imdb_ge_df_tops.shape

((400, 34), (400, 34), (400, 34), (400, 34))

In [320]:
imdb_kr_df_tops.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,Unnamed: 0,title,year,country
971,0.000000,0.037582,0.000000,0.000000,0,0,0.000000,0,0,0,...,0.477371,0.034577,0.000000,0.000000,0.000000,0.240565,971,1408,2007,South Korea
14125,0.052527,0.063921,0.026838,0.111481,0,0,0.000000,0,0,0,...,0.000000,0.000000,0.089186,0.000000,0.155246,0.000000,14125,Alvin and the Chipmunks,2007,South Korea
14758,0.000000,0.000000,0.000000,0.156653,0,0,0.040150,0,0,0,...,0.000000,0.031039,0.157441,0.051157,0.321034,0.000000,14758,American Gangster,2007,South Korea
20642,0.015923,0.108853,0.000000,0.000000,0,0,0.070073,0,0,0,...,0.000000,0.000000,0.000000,0.068219,0.086692,0.000000,20642,August Rush,2007,South Korea
24826,0.000000,0.000000,0.000000,0.103414,0,0,0.000000,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.107138,0.122606,24826,Because I Said So,2007,South Korea


In [302]:
#topic scores for top movies for each country
kr_topic = imdb_kr_df_tops[imdb_kr_df_tops.columns[:30]] #score for each topic is in column 0~29
kr_topic_score= kr_topic.sum(axis=0)
us_topic = imdb_us_df_tops[imdb_us_df_tops.columns[:30]]
us_topic_score= us_topic.sum(axis=0)
uk_topic = imdb_uk_df_tops[imdb_uk_df_tops.columns[:30]]
uk_topic_score= uk_topic.sum(axis=0)
ge_topic = imdb_ge_df_tops[imdb_ge_df_tops.columns[:30]]
ge_topic_score= ge_topic.sum(axis=0)
len(kr_topic_score), len(us_topic_score), len(uk_topic_score), len(ge_topic_score), 

(30, 30, 30, 30)

In [305]:
#Create dataframe for visualization (since 2007, full year)
country_us = ['United States'] *30
country_ge = ['Germany']*30
country_uk = ['United Kingdom']*30
country_kr = ['South Korea'] *30
topics =['love, family', 'love, relationships', 'earth, nature', 'friendships, relationships', 'crime, mistery', 'society, culture', 
         'home', 'money', 'gangs, drug, police', 'documentary, interview', 'ight life, enjoyment', 'music', 'war', 
         'school, college', 'fantasy, fairy tale', 'cowboys and indians', 'photagraphy, design', 'comedy', 'spies, terrorism', 
         'crime, police, underworld', 'places, nature, scenery', 'historical, costumes','town','hollywood, stars', 'press, politics','farming, country side',
         'fantasy, christmas', 'magic, myths', 'ships, sailing, pirates', 'mixed']
data_us = {'Country':country_us, 'Topic':topics, 'Topic_Score':us_topic_score}
data_ge = {'Country':country_ge, 'Topic':topics, 'Topic_Score':ge_topic_score}
data_uk = {'Country':country_uk, 'Topic':topics, 'Topic_Score':uk_topic_score}
data_kr = {'Country':country_kr, 'Topic':topics, 'Topic_Score':kr_topic_score} 

us = pd.DataFrame(data_us)
ge = pd.DataFrame(data_ge)
uk = pd.DataFrame(data_uk)
kr = pd.DataFrame(data_kr)

frames =[us,ge, uk, kr]
viz_data_fullyear = pd.concat(frames)

In [306]:
viz_data_fullyear.head()

,Country,Topic,Topic_Score
0,United States,"love, family",2.869794
1,United States,"love, relationships",7.037957
2,United States,"earth, nature",1.967613
3,United States,"friendships, relationships",9.036039
4,United States,"crime, mistery",2.627311


In [311]:
#filtering global top10 topics for each country : topic 5, 28, 21, 7,24, 11, 20, 29, 13, 25
global_topics = ['society, culture', 'ships, sailing, pirates', 'historical, costumes','money', 'press, politics', 'music', 
                'places, nature, scenery', 'mixed', 'school, college', 'farming, country side']
viz_data_global_topic = viz_data_fullyear[viz_data_fullyear['Topic'].isin(global_topics)]
viz_data_global_topic.head()

,Country,Topic,Topic_Score
5,United States,"society, culture",26.282347
7,United States,money,21.976539
11,United States,music,16.648588
13,United States,"school, college",11.899355
20,United States,"places, nature, scenery",12.982491


In [312]:
#save data as csv for visualization
viz_data_fullyear.to_csv("viz_fullyear.csv", index=False) #30 topics for each country
viz_data_global_topic.to_csv("viz_global_topic.csv", index=False) #global top10 topics for each country

## Temporal Behavior of Topic Distribution

Temporal behavior/evolution of topic distribution can be studied in two ways : (i) top 10 topic evolution with time in each country, (ii) temporal evolution of global top 10 topics in each country. If the top 10 topics in each country are quite different from global top 10, approach (i) should be meaningful(because we can observe what is the emerging/prominent topic and what topic is decaying recently for a specific country), while top 10 topics in each country are similar to global top 10, approach (ii) may be meaningful. Let's check if top 10 topics in each country are quite different from global top 10 topics.

In [313]:
#top 10 topics in each country
us_top = us.sort_values('Topic_Score', axis=0, ascending=False, inplace=False)
ge_top = ge.sort_values('Topic_Score', axis=0, ascending=False, inplace=False)
uk_top = uk.sort_values('Topic_Score', axis=0, ascending=False, inplace=False)
kr_top = kr.sort_values('Topic_Score', axis=0, ascending=False, inplace=False)

In [317]:
us_top10 = us_top['Topic'].tolist()
ge_top10 = ge_top['Topic'].tolist()
uk_top10 = uk_top['Topic'].tolist()
kr_top10 = kr_top['Topic'].tolist()

In [318]:
print "US top 10 topic: ", us_top10[:10]
print "GE top 10 topic: ", ge_top10[:10]
print "UK top 10 topic: ", uk_top10[:10]
print "KR top 10 topic: ", kr_top10[:10]

US top 10 topic:  ['society, culture', 'ships, sailing, pirates', 'money', 'historical, costumes', 'music', 'places, nature, scenery', 'mixed', 'press, politics', 'school, college', 'farming, country side']
GE top 10 topic:  ['society, culture', 'ships, sailing, pirates', 'money', 'historical, costumes', 'places, nature, scenery', 'music', 'mixed', 'press, politics', 'school, college', 'friendships, relationships']
UK top 10 topic:  ['society, culture', 'money', 'ships, sailing, pirates', 'historical, costumes', 'music', 'places, nature, scenery', 'press, politics', 'mixed', 'school, college', 'farming, country side']
KR top 10 topic:  ['money', 'society, culture', 'ships, sailing, pirates', 'historical, costumes', 'music', 'mixed', 'press, politics', 'friendships, relationships', 'places, nature, scenery', 'school, college']


As we can see above results, top 10 topics of each country are almost similar to globaltop 10 topics. Thus, temporal behavior of global top 10 topics should be able to represent(approximately) the temporal, major topic distribution for each country.

In [323]:
#yearly topic score for each country
years = range(2007, 2015)
kr_yearly_topic_score =[]
for year in years:
    kr_df_year = (imdb_kr_df_tops[imdb_kr_df_tops.year == year])[:30]
    kr_yearly_topic_score.append(kr_df_year.sum(axis=0))


30

## Topic Distribution with Origin of Movies

Now we are going to investigate the topic distribution of popular movies with origin of movies. For that purpose, create a dataframe "imdb_meta_top_df" containing origin of movies for popular movies first. 

In [73]:
imdb_meta_top_df = pd.DataFrame()

for year in range(2001, 2015):
    meta = get(imdb_meta_df, year)
    top = get(topmovies_df, year)
    #remove date from imdb title
    titles = meta.title.apply(lambda x : x.split('(')[0].strip())
    meta = meta[titles.isin(top.movie.tolist())]
    imdb_meta_top_df = pd.concat([imdb_meta_top_df, meta])

In [75]:
imdb_meta_top_df.head(5)

,Unnamed: 0,title,year
994,994,15 Minutes,2001
2069,2069,3000 Miles to Graceland,2001
3342,3342,A Beautiful Mind,2001
5579,5579,A Knight's Tale,2001
12647,12647,Ali,2001


In [136]:
for year in range(2001, 2015):
    for country in ['Germany','United States', 'United Kingdom', 'South Korea', 'Japan', 'Hong Kong']:
        print country
        print year
        print get(imdb_merged_df,year, country).shape[0]
        print '================================'

Germany
2001
77
United States
2001
183
United Kingdom
2001
0
South Korea
2001
0
Japan
2001
0
Hong Kong
2001
0
Germany
2002
83
United States
2002
215
United Kingdom
2002
97
South Korea
2002
0
Japan
2002
40
Hong Kong
2002
48
Germany
2003
90
United States
2003
190
United Kingdom
2003
104
South Korea
2003
0
Japan
2003
36
Hong Kong
2003
43
Germany
2004
110
United States
2004
227
United Kingdom
2004
151
South Korea
2004
0
Japan
2004
37
Hong Kong
2004
38
Germany
2005
91
United States
2005
213
United Kingdom
2005
135
South Korea
2005
0
Japan
2005
38
Hong Kong
2005
41
Germany
2006
112
United States
2006
229
United Kingdom
2006
174
South Korea
2006
0
Japan
2006
33
Hong Kong
2006
41
Germany
2007
124
United States
2007
225
United Kingdom
2007
189
South Korea
2007
74
Japan
2007
29
Hong Kong
2007
45
Germany
2008
105
United States
2008
220
United Kingdom
2008
192
South Korea
2008
68
Japan
2008
31
Hong Kong
2008
43
Germany
2009
102
United States
2009
206
United Kingdom
2009
140
South Korea
2009
63
Jap

In [ ]:
with open(dropbox + "imdb/countries.list") as f:
    countries = f.readlines()
    
imdb_meta_top_df['origin'] = pd.Series(index=imdb_meta_top_df.index)
for i,movie in enumerate(countries):
    if movie[0] == '"':
        continue
    split = movie.split('\t')
    title = split[0]
    idx = imdb_meta_top_df[imdb_meta_top_df['title']== title].index
    #if the title is in our dataframe add the country
    if len(idx > 0):
        imdb_meta_top_df.loc[[idx[0]],['origin']]= split[len(split)-1].replace('\n','')

In [ ]:
imdb_meta_top_df.to_csv(dropbox + 'imdb_meta_top_df.csv')

In [80]:
imdb_meta_top_df = pd.read_csv(dropbox + 'imdb_meta_top_df.csv')
imdb_meta_top_df.head() #imdb_meta_top_df = data arranged for top movies with origin of movies

,Unnamed: 0,Unnamed: 0.1,title,year,origin
0,994,994,15 Minutes (2001),2001,Germany
1,2069,2069,3000 Miles to Graceland (2001),2001,USA
2,3342,3342,A Beautiful Mind (2001),2001,USA
3,5579,5579,A Knight's Tale (2001),2001,USA
4,12647,12647,Ali (2001),2001,USA


In [ ]:
imdb_meta_top_df.head()

In [ ]:
imdb_topic_labels = {0:'love, family', 
                1:'love, relationships', 
                2:'earth, nature', 
                3:'friendships, relationships', 
                4:'crime, mistery', 
                5:'society, culture', 
                6:'home',
                7:'money',
                8:'gangs, drug, police',
                9:'documentary, interview',
                10:'night life, enjoyment',
                11:'music',
                12:'war',
                13:'school, college',
                14:'fantasy, fairy tale',
                15:'cowboys and indians',
                16:'photagraphy, design',
                17:'comedy',
                18:'spies, terrorism',
                19:'crime, police, underworld',
                20:'places, nature, scenery',
                21:'historical, costumes',
                22:'town',
                23:'hollywood, stars',
                24:'press, politics',
                25:'farming, country side',
                26:'fantasy, christmas',
                27:'magic, myths',
                28:'ships, sailing, pirates',
                29:'mixed'}